In [1]:
import pandas as pd

In [2]:
ASIGNATURAS = {
    'PROGRAMACION': 2511,
    'INGLES': 2512,
    'COACH': 2513
}

In [3]:
CONSULTA = pd.read_csv('./INPUT/CHEQUEO/SISTEMA.csv', sep=';', encoding='latin-1', skiprows=1)
REAL = pd.read_excel('./INPUT/CHEQUEO/CONSOLIDADO_MATRICULAS_2022-06-08.xlsx', engine='openpyxl')

In [4]:
rta = {
    'CODIGO': [],
    'NOMBRE': [],
    'PROGRAMACION_REAL': [],
    'PROGRAMACION_SA': [],
    'PROGRAMACION_COINCIDE': [],
    'INGLES_REAL': [],
    'INGLES_SA': [],
    'INGLES_COINCIDE': [],
    'COACH_REAL': [],
    'COACH_SA': [],
    'COACH_COINCIDE': [],
}
codigos = CONSULTA['ESTUDIANTE'].unique()
for codigo in codigos:
    dfEst = CONSULTA[CONSULTA['ESTUDIANTE'] == codigo]
    dfEstReal = REAL[REAL['CODIGO'] == codigo]
    rta['CODIGO'].append(codigo)
    rta['NOMBRE'].append(dfEst['APELLIDOS'].iloc[0] + dfEst['NOMBRES'].iloc[0])
    for asignatura, codAsignatura in ASIGNATURAS.items():
        grupoReal = dfEstReal[asignatura].iloc[0]
        rta[asignatura + '_REAL'].append(grupoReal)
        temp = dfEst[dfEst['ASIGNATURA'] == codAsignatura]
        if len(temp) == 0:
            rta[asignatura + '_SA'].append('NO MATRICULADO')
            rta[asignatura + '_COINCIDE'].append('N/A')
        else:
            grupo = temp['GRUPO'].iloc[0]
            rta[asignatura + '_SA'].append(grupo)
            rta[asignatura + '_COINCIDE'].append(True if grupoReal == grupo else False)
resultados = pd.DataFrame(rta)

In [5]:
from datetime import date
resultados.to_excel('./OUTPUT/CHEQUEO/REVISIÓN_' + str(date.today()) + '_6.xlsx', index = False)